<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/pytorch-with-examples/07_dynamic_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

#PyTorch: Control Flow + Weight Sharing
To showcase the power of PyTorch dynamic graphs, we will implement a very strange model: a third-fifth order polynomial that on each forward pass chooses a random number between 3 and 5 and uses that many orders, resuing the same weights multiple times to compute the forth and fifth order. 

In [10]:
import random 
import torch 
import math 

class DynamicNet(torch.nn.Module):
  def __init__(self):
    """
    In the constructor we instantiate five parameters and assign them as members. 
    """
    super().__init__()
    self.a = torch.nn.Parameter(torch.randn(()))
    self.b = torch.nn.Parameter(torch.randn(()))
    self.c = torch.nn.Parameter(torch.randn(()))
    self.d = torch.nn.Parameter(torch.randn(()))
    self.e = torch.nn.Parameter(torch.randn(()))

  def forward(self, x):
    """
    For the forward pass of the model, we randomly choose either 4 or 5
    and reuse the e parameter to compute the contribution of these orders. 

    Since each forward pass builds a dynamic computation graph, we can use normal
    Python control-flow operators like loops or conditional statements when 
    defining the forward pass of the model. 

    Here we also see that it is perfectly safe to reuse the same parameter many
    times when defining a computational graph. 
    """
    y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
    for exp in range(4, random.randint(4,6)):
      y = y + self.e * x ** exp 
    return y 

  def string(self):
    """
    Just like any class in Python, you can also define custom method on 
    PyTorch modules 
    """
    x = f'y = ' 
    y = f'{" %.4f + "}'%self.a.item() 
    z = f'{" %.4f x + "}'%self.b.item()  
    w = f'{" %.4f x^2 + "}'%self.c.item()  
    a = f'{" %.4f x^3 +"}'%self.d.item()
    b = f'{" %.4f x^4 ?"}'%self.e.item()
    c = f'{" %.4f x^5 ?"}'%self.e.item()
    return x+y+z+w+a+b+c

# Create Tensors to hold input and outputs. 
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above 
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum 
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(3000):
   # Forward pass: Compute predicted y by passing x to the model 
   y_pred = model(x)
   # Compute and print loss 
   loss = criterion(y_pred, y)
   if t % 100 == 99:
     print(t, loss.item())

   # Zero gradients, perform a backward pass, and update the weights. 
   optimizer.zero_grad()
   loss.backward()
   optimizer.step()


print(f'Result: {model.string()}')



99 21088106.0
199 243403.953125
299 1509.9053955078125
399 2187.60009765625
499 2908.666748046875
599 1204.462646484375
699 1166.0146484375
799 1121.0164794921875
899 1078.45166015625
999 1027.33837890625
1099 991.9153442382812
1199 961.5921020507812
1299 939.3413696289062
1399 898.3304443359375
1499 854.445068359375
1599 822.7882690429688
1699 821.8604125976562
1799 760.6082763671875
1899 802.4473876953125
1999 711.8200073242188
2099 661.5264892578125
2199 650.8818359375
2299 638.4863891601562
2399 597.8087768554688
2499 575.25439453125
2599 558.0422973632812
2699 561.4852294921875
2799 510.6127014160156
2899 515.29443359375
2999 478.595458984375
Result: y =  0.4050 +  0.3032 x +  -0.0676 x^2 +  -0.0131 x^3 + -0.0005 x^4 ? -0.0005 x^5 ?
